<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_1_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción
gpu----------

Por lo general, las computadoras se utilizan para compilar y analizar los resultados de encuestas y estudios de opinión.

El siguiente cuaderno calcula la **moda** de los **N** valores de los elementos de un vector (vector_resultados), lo hace en forma secuencial y utilizando el procesador CPU. Cada elemento del vector se inicializa con resultados aleatorios de puntajes (numeros enteros entre 0 y 9).

El algoritmo se basa en un ejemplo práctico del libro "C/C++ Cómo Programar"[3] 

Su objetivo es aprender a utilizar Python[2] en la plataforma Colab [1] y la programación secuencial.

#2. Armado de ambiente

Instalación de modulo CUDA  de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 14.9MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=3b27c08ebc6d21688b77c86ebf689f5c0fd55a1af43df08e8ec4e55e81f3a662
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=2c584e05815c67bea2713045eb4b76449d4879db9f8b760606f74af7ec3c7466
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo


In [166]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   50#@param {type: "number"}

# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0


# CPU - Defino la memoria de los vectores en cpu.
x_cpu = numpy.random.randn( cantidad_N )
x_cpu = x_cpu.astype( numpy.float32() )


#tiempo_ini_cpu = datetime.now()

r_cpu = numpy.empty_like( x_cpu )

# CPU - reservo la memoria GPU.
x_gpu = cuda.mem_alloc( x_cpu.nbytes )

# GPU - Copio la memoria al GPU.
cuda.memcpy_htod( x_gpu, x_cpu )

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_axpy( int n, float *X )
{
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  if( idx < n )
  {
      if(X[idx] == 0)
        X[idx] = 0;
      else
        X[idx]  = X[idx] / X[idx];
  }
}
""") 
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_axpy")

tiempo_gpu = datetime.now()

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel( numpy.int32(cantidad_N), x_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

tiempo_gpu = datetime.now() - tiempo_gpu

# GPU - Copio el resultado desde la memoria GPU.
cuda.memcpy_dtoh( r_cpu, x_gpu )


# CPU - Informo el resutlado.
print( "------------------------------------")
print( "X: " )
print( x_cpu )
print( "------------------------------------")
print( "R: " )
print( r_cpu )


tiempo_total = datetime.now() - tiempo_total

print( "Cantidad de elementos: ", cantidad_N )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_en_ms( tiempo_total ), "[ms]" )
print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )

Thread x:  256 , Bloque x: 1
------------------------------------
X: 
[ 0.14327128 -1.1691452  -1.0609384   1.0811981  -0.69590634  0.2138904
  0.01431662  0.14744574  0.08865719  1.0298524  -0.36717856 -1.1830544
 -0.44746318 -0.47035813  0.29482543 -0.7107522   0.0579996  -0.55715406
  0.4021698  -1.3428118  -0.26230258 -1.2498338   1.5534211   1.2672026
 -0.8454386  -0.3852646  -0.3396521   1.5116189  -0.44174123  2.0967224
  0.9695537   1.2783296  -0.12307635 -0.7236193   0.7360623  -0.3916892
  0.33471447  1.1447508   0.57564414  0.20260687  0.472679    1.1229506
 -1.0026025  -0.7930292   0.17806065 -3.2498894  -0.10784678 -1.0287619
  0.10549242  1.5348873 ]
------------------------------------
R: 
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Cantidad de elementos:  50
Thread x:  256 , Bloque x: 1
Tiempo CPU:  6.485 [ms]
Tiempo GPU:  1.343 [ms]


#4. Tabla de pasos

Paso | Procesador | Funcion | Detalle
------------ | ------------ | ------------- | -------------
1 | CPU | @param | Lectura del tamaño de vector de Colab.
2 | CPU | import | Importa los módulos para funcionar.
3 | CPU | datetime.now() | Toma el tiempo actual.
4 | CPU | np.random.randint(0, 10, size = cantidad_elementos) | Inicializa el vector _vector_resultados_ con puntajes (de 0 a 9) aleatorios en cada elemento, en una cantidad de elementos ingresada en el paso 1.
5 | CPU | [0 for i in range(10)] | inicializo el vector frecuencia a 0, este vector representará en cada posición el valor del puntaje (entre 0 y 9) y su contenido representará la cantidad de apariciónes en el vector resultados. Por esta razon es necesario inicializarlo a cero, para luego sumar las frecuencias.
6 | CPU | print | muestro los valores del vector_resultados
7 | CPU | bucle for | contabilizo las frecuencias de puntajes en el vector_resultados.
8 | CPU | print | imprimo los resultados del vector frecuencia
9 | CPU | bucle for | obtengo el valor de la moda y la cantidad de repeticiones.
10 | CPU | print | muestro los resultados
11 | CPU | print | informo tiempos de ejecución.

#5. Conclusiones

TODO: Preparar y correr cuaderno en gpu y realizar comparación.....

#6. Bibliografía

[1] MARKDOWN SYNTAX Colab: PDF

[2] Introducción a Python: Página Colab

[3] Como Programar en C C++ y Java 4ta Edición Harvey M. Deitel & Paul J. Deitel 

[4] Biblioteca BLAS: Referencia

[5] Documentación PyCUDA: WEB

[6] Repositorio de PyCUDA: WEB